In [2]:
# import libraries
import numpy as np
import pandas as pd
import random 
import math
import time 
import plotly.graph_objects as go
import datetime
import operator
import warnings
from IPython.display import clear_output
warnings.filterwarnings("ignore")

In [3]:
# import data (make sure to update daily)

confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveries_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/09-08-2020.csv')
us_medical_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/09-08-2020.csv')
apple_mobility = pd.read_csv('https://covid19-static.cdn-apple.com/covid19-mobility-data/2016HotfixDev10/v3/en-us/applemobilitytrends-2020-09-07.csv')

In [4]:
# set columns from dataframe
cols = confirmed_df.keys()   # pandas Index
# get all the dates for the outbreak
confirmed = confirmed_df.iloc[:,4:-1]   #this will return all column keys from the fifth column to the last
deaths = deaths_df.iloc[:, 4:-1]
recoveries = recoveries_df.iloc[:, 4:-1]
dates = confirmed.keys() # get dates

In [5]:
# helper functions for daily increases and moving averages

# define a support function to get daily increases
def daily_increase(data):
    # initialize empty list to track daily increases
    d = []
    # loop througth data and track increase in data
    for i in range(len(data)):
        # if statment for first day
        if i == 0:
            d.append(data[0])
        # otherwise we collect given day amount subtracted by yesterdays amount
        else:
            d.append(data[i]-data[i-1])
    # return our list
    return d
    
    
    
# now we define a moving average function
# data is our input data list
# window_size is the days for calculating average (7 for a week)
def moving_average(data, window_size):
    # initialize empty list
    moving_average = []
    # loop through list
    for i in range(len(data)):
        # we have a condition at the end of the list, we cant add values that are larger than 
        # the indices of the list, hence we make a condition with if statememt
        if i + window_size < len(data):
            moving_average.append(np.mean(data[i:i+window_size]))
        else:
            moving_average.append(np.mean(data[i:len(data)]))
    return moving_average


In [6]:
# Main Function
def country_plot(country_name):
    
    # if statement so that the input is a country name that exists in our dataset.
    if country_name in countries:
    
    # first we get all the country's cases, deaths, and recoveries
        country_cases = []
        country_deaths = []
        country_recovered = []
        country_active = []
        mortality_rate = []
    
        for i in dates:
            country_cases.append(confirmed_df[confirmed_df['Country/Region']==country_name][i].sum())
            country_deaths.append(deaths_df[deaths_df['Country/Region']==country_name][i].sum())
            country_recovered.append(recoveries_df[recoveries_df['Country/Region']==country_name][i].sum())
            mortality_rate.append((deaths_df[deaths_df['Country/Region']==country_name][i].sum()/
                                 confirmed_df[confirmed_df['Country/Region']==country_name][i].sum())*100.0)
    
    
        for i in range(len(country_cases)):
            country_active.append(country_cases[i] - country_deaths[i] - country_recovered[i])
            
        # lets get some values
        window = 7 

        # world cases 
        country_daily_increase = daily_increase(country_cases)
        country_daily_increase_average = moving_average(country_daily_increase, window)
        country_confirmed_average = moving_average(country_cases, window)

        # deaths
        country_deaths_increase = daily_increase(country_deaths)
        country_deaths_confirmed_average = moving_average(country_deaths, window)
        country_deaths_increase_average = moving_average(country_deaths_increase, window)

        # recovered
        country_recovered_increase = daily_increase(country_recovered)
        country_recovered_increase_average = moving_average(country_recovered_increase, window)
        country_recovered_confirmed_average = moving_average(country_recovered, window)

        # active cases
        country_active_increase = daily_increase(country_active)
        country_active_confirmed_average = moving_average(country_active, window)
        country_active_increase_average = moving_average(country_active_increase, window)

        # first graph 
        fig = go.Figure()

        # world cases line
        fig.add_trace(go.Scatter(x = dates,
                             y = country_cases,
                             mode = 'lines',
                             name = 'number of world cases in ' + country_name))

        # world moving average
        fig.add_trace(go.Scatter(x = dates,
                             y = country_confirmed_average,
                             name = 'moving average',
                             line = dict(color = 'firebrick', dash = 'dash')))

        # Edit the layout
        fig.update_layout(title= country_name + ' Total Number Of Confirmed Cases',
                       xaxis_title='Date',
                       yaxis_title='Number of Cases')

        fig.show()

        # Graph 2

        # create traces
        fig2 = go.Figure()

        # country deaths line
        fig2.add_trace(go.Scatter(x = dates,
                                  y = country_deaths,
                                  mode = 'lines',
                                  name = 'total deaths in ' + country_name))

        # deaths moving average
        fig2.add_trace(go.Scatter(x = dates,
                                  y = country_deaths_confirmed_average,
                                  name = 'moving average',
                                  line = dict(color = 'firebrick', dash = 'dash')))

        # Edit the layout
        fig2.update_layout(title= country_name + ' Total Number Of Deaths',
                           xaxis_title='Date',
                           yaxis_title='Number of Deaths')

        fig2.show()
        
        # figure 3
        fig3 = go.Figure()

        # country recovered line
        fig3.add_trace(go.Scatter(x = dates,
                                 y = country_recovered,
                                 mode = 'lines',
                                 name = 'total recovered in ' + country_name))

        # recovered moving average
        fig3.add_trace(go.Scatter(x = dates,
                                 y = country_recovered_confirmed_average,
                                 name = 'moving average',
                                 line = dict(color = 'firebrick', dash = 'dash')))

        # Edit the layout
        fig3.update_layout(title= country_name + ' Total Number Of Recoveries',
                           xaxis_title='Date',
                           yaxis_title='Number of Recovered')

        fig3.show()
        
        # figure 4
        fig4 = go.Figure()

        #  active line
        fig4.add_trace(go.Scatter(x = dates,
                                 y = country_active,
                                 mode = 'lines',
                                 name = 'total active in ' + country_name ))

        # moving average
        fig4.add_trace(go.Scatter(x = dates,
                                 y = country_active_confirmed_average,
                                 name = 'moving average',
                                 line = dict(color = 'firebrick', dash = 'dash')))

        # Edit the layout
        fig4.update_layout(title= country_name + ' Total Number Of Active Cases',
                           xaxis_title='Date',
                           yaxis_title='Number of Active Cases')

        fig4.show()
        
        # figure 5
        fig5 = go.Figure()

        # daily increase bar chart
        fig5.add_trace(go.Bar(x = dates,
                                 y = country_daily_increase,
                                 name = 'Daily Cases in ' + country_name))

        # daily increase moving average
        fig5.add_trace(go.Scatter(x = dates,
                                 y = country_daily_increase_average,
                                 name = 'moving average',
                                 line = dict(color = 'firebrick', dash = 'dash')))

        # Edit the layout
        fig5.update_layout(title= country_name + ' Daily Increases in Confirmed Cases',
                           xaxis_title='Date',
                           yaxis_title='Number of Confirmed Cases')

        fig5.show()
        
        # figure 6
        fig6 = go.Figure()

        # daily deaths bar chart
        fig6.add_trace(go.Bar(x = dates,
                                 y = country_deaths_increase,
                                 name = country_name + ' Daily Deaths'))

        # daily deaths increase moving average
        fig6.add_trace(go.Scatter(x = dates,
                                 y = country_deaths_increase_average,
                                 name = 'moving average',
                                 line = dict(color = 'firebrick', dash = 'dash')))

        # Edit the layout
        fig6.update_layout(title= country_name + ' Daily Increases in Deaths',
                           xaxis_title='Date',
                           yaxis_title='Number of Deaths')

        fig6.show()
        
        # figure 7
        fig7 = go.Figure()

        # daily recovered bar chart
        fig7.add_trace(go.Bar(x = dates,
                                 y = country_recovered_increase,
                                 name = 'Daily Recoveries in ' + country_name))

        # daily recovred increase moving average
        fig7.add_trace(go.Scatter(x = dates,
                                 y = country_recovered_increase_average,
                                 name = 'moving average',
                                 line = dict(color = 'firebrick', dash = 'dash')))

        # Edit the layout
        fig7.update_layout(title= country_name + ' Daily Increases in Recoveries',
                           xaxis_title='Date',
                           yaxis_title='Number of recoveres')

        fig7.show()
        
        # figure 8
        fig8 = go.Figure()

        # daily active bar chart
        fig8.add_trace(go.Bar(x = dates,
                             y = country_active_increase,
                             name = 'Daily Active in ' + country_name))

        # daily active increase moving average
        fig8.add_trace(go.Scatter(x = dates,
                                 y = country_active_increase_average,
                                 name = 'moving average',
                                 line = dict(color = 'firebrick', dash = 'dash')))

        # Edit the layout
        fig8.update_layout(title= country_name + ' Daily Increases in Active Cases',
                           xaxis_title='Date',
                           yaxis_title='Number of active')

        fig8.show()

        # figure 9 - log plots
        fig9 = go.Figure()

        # cases log line
        fig9.add_trace(go.Scatter(x = dates,
                                 y = np.log10(country_cases),
                                 mode = 'lines',
                                 name = 'log of number of cases in ' + country_name))


        # Edit the layout
        fig9.update_layout(title='Log of # of Coronavirus Cases Over Time in ' + country_name,
                           xaxis_title='Date',
                           yaxis_title='Log of Number of Cases')

        fig9.show()
        
        # figure 10 log of deaths
        fig10 = go.Figure()

        # total deaths log line
        fig10.add_trace(go.Scatter(x = dates,
                                 y = np.log10(country_deaths),
                                 mode = 'lines',
                                 name = 'Log of number of Deaths in ' + country_name))


        # Edit the layout
        fig10.update_layout(title='Log of Deaths of Coronavirus Cases Over Time in ' + country_name,
                           xaxis_title='Date',
                           yaxis_title='Log of Number of Deaths')

        fig10.show()

        # figure 11 - log recovered
        fig11 = go.Figure()

        # total recovered log line
        fig11.add_trace(go.Scatter(x = dates,
                                 y = np.log10(country_recovered),
                                 mode = 'lines',
                                 name = 'log of number of recovered in ' + country_name))


        # Edit the layout
        fig11.update_layout(title='Log of Recovered of Coronavirus Cases Over Time in ' + country_name,
                           xaxis_title='Date',
                           yaxis_title='Log of Number of Recoveries')

        fig11.show()
        
        # figure 12 - log of active 
        fig12 = go.Figure()

        # total active log line
        fig12.add_trace(go.Scatter(x = dates,
                                 y = np.log10(country_active),
                                 mode = 'lines',
                                 name = 'log of number of active cases in ' + country_name))


        # Edit the layout
        fig12.update_layout(title='Log of Number of Active Coronavirus Cases Over Time in ' + country_name,
                           xaxis_title='Date',
                           yaxis_title='Log of Number of Active Cases')

        fig12.show()
        
        # mortality rate plot
        fig13 = go.Figure()

        # total active log line
        fig13.add_trace(go.Scatter(x = dates,
                                 y = mortality_rate,
                                 mode = 'lines',
                                 name = 'mortality rate in ' + country_name))


        # Edit the layout
        fig13.update_layout(title='Mortality Rate in ' + country_name,
                           xaxis_title='Date',
                           yaxis_title='Mortality Rate (%)')

        fig13.show()

    else:
        print("data does not have entered country")


In [11]:
# lets take a look at what unique countries we have in our data
countries = []
countries = confirmed_df['Country/Region'].unique()   
my_countries = ', \n'.join(countries)
type(list(countries))

list

In [8]:
# Using input so user can select a country to check data.

# Initialize a string for our input
country_choice = ''

# instructions
print('Enter a country name too see Covid-19 plots. Enter "break" if you want to stop')
print('\n below is a list of available countries')
print(my_countries)
country_choice = input('Please enter a country: ')
while country_choice != 'break':
    country_plot(country_choice)
    country_choice = input('Want to see another country? Here are your options: ' + my_countries + '\n enter a new country here: ')

    if True:
        clear_output(wait=True)

Enter a country name too see Covid-19 plots. Enter "break" if you want to stop

 below is a list of available countries
Afghanistan, 
Albania, 
Algeria, 
Andorra, 
Angola, 
Antigua and Barbuda, 
Argentina, 
Armenia, 
Australia, 
Austria, 
Azerbaijan, 
Bahamas, 
Bahrain, 
Bangladesh, 
Barbados, 
Belarus, 
Belgium, 
Belize, 
Benin, 
Bhutan, 
Bolivia, 
Bosnia and Herzegovina, 
Botswana, 
Brazil, 
Brunei, 
Bulgaria, 
Burkina Faso, 
Burma, 
Burundi, 
Cabo Verde, 
Cambodia, 
Cameroon, 
Canada, 
Central African Republic, 
Chad, 
Chile, 
China, 
Colombia, 
Comoros, 
Congo (Brazzaville), 
Congo (Kinshasa), 
Costa Rica, 
Cote d'Ivoire, 
Croatia, 
Cuba, 
Cyprus, 
Czechia, 
Denmark, 
Diamond Princess, 
Djibouti, 
Dominica, 
Dominican Republic, 
Ecuador, 
Egypt, 
El Salvador, 
Equatorial Guinea, 
Eritrea, 
Estonia, 
Eswatini, 
Ethiopia, 
Fiji, 
Finland, 
France, 
Gabon, 
Gambia, 
Georgia, 
Germany, 
Ghana, 
Greece, 
Grenada, 
Guatemala, 
Guinea, 
Guinea-Bissau, 
Guyana, 
Haiti, 
Holy See, 
Honduras

In [38]:
# List of countries for drop down menu
countries = list(confirmed_df['Country/Region'].unique())

# make up a list for our options
options_lst = []
for i in np.arange(0,len(countries), 1):
    options_lst.append({'Label': countries[i], 'value': countries[i]})

In [40]:
options_lst

[{'Label': 'Afghanistan', 'value': 'Afghanistan'},
 {'Label': 'Albania', 'value': 'Albania'},
 {'Label': 'Algeria', 'value': 'Algeria'},
 {'Label': 'Andorra', 'value': 'Andorra'},
 {'Label': 'Angola', 'value': 'Angola'},
 {'Label': 'Antigua and Barbuda', 'value': 'Antigua and Barbuda'},
 {'Label': 'Argentina', 'value': 'Argentina'},
 {'Label': 'Armenia', 'value': 'Armenia'},
 {'Label': 'Australia', 'value': 'Australia'},
 {'Label': 'Austria', 'value': 'Austria'},
 {'Label': 'Azerbaijan', 'value': 'Azerbaijan'},
 {'Label': 'Bahamas', 'value': 'Bahamas'},
 {'Label': 'Bahrain', 'value': 'Bahrain'},
 {'Label': 'Bangladesh', 'value': 'Bangladesh'},
 {'Label': 'Barbados', 'value': 'Barbados'},
 {'Label': 'Belarus', 'value': 'Belarus'},
 {'Label': 'Belgium', 'value': 'Belgium'},
 {'Label': 'Belize', 'value': 'Belize'},
 {'Label': 'Benin', 'value': 'Benin'},
 {'Label': 'Bhutan', 'value': 'Bhutan'},
 {'Label': 'Bolivia', 'value': 'Bolivia'},
 {'Label': 'Bosnia and Herzegovina', 'value': 'Bosnia

In [48]:
def plotgraph():
    fig = go.Figure()
    # world cases line
    fig.add_trace(go.Scatter(x = dates,
                             y = world_cases,
                             mode = 'lines',
                             name = 'number of world cases in ' + country_name))

     # world moving average
    fig.add_trace(go.Scatter(x = dates,
                             y = world_confirmed_average,
                             name = 'moving average',
                             line = dict(color = 'firebrick', dash = 'dash')))

     # Edit the layout
    fig.update_layout(title= ' Total Number Of Confirmed Cases',
                       xaxis_title='Date',
                       yaxis_title='Number of Cases')
    return fig
